# MOVIE LENS 10M

In [77]:
import pandas as pd
import numpy as np
import json

In [78]:
def load_ratings():

    return pd.read_csv(
        './dataset/ratings_cortito.dat',
        sep='::',
        header=None,
        names=['userId', 'movieId', 'rating', 'timestamp'])
      
def load_movies():

    return pd.read_csv(
        './dataset/movies.dat',
        sep='::',
        header=None,
        names=['movieID', 'Title', 'Genre'])

def get_movies_id_list():

        movies = pd.read_csv(
        './dataset/movies.dat',
        sep='::',
        header=None,
        names=['movieID', 'Title', 'Genre'])
        retorno = movies.values[:, 0]
        return retorno

def get_global_ratings_statistics():
    ratings = load_ratings()
    return np.mean(ratings["rating"]), np.std(ratings["rating"])

def get_filler_ratings_statistics(filler_items):
    return 5,5 #TODO

def get_windows(n_items, n_windows=40):
    """
    Returns the items partitions made.
    Both first item and last item are included.

    Returns:
        List of tuples, where the tuples are 
        (first_item_window_INCLUDED, last_item_window_EXCLUDED)
    """

    Q, q = divmod(n_items, n_windows)
    index = 0
    windows = []

    for j in range(n_windows):

        if j < q: #Cambiado de su paper porque no sense el =
          n_items_window = Q + 1

        else:
          n_items_window = Q

        windows.append((index, index + n_items_window))
        index = index + n_items_window 

    return windows
    

def obtain_array_user(ratings_user, J=40):
    """Entrada: array con las ratings
    """

    movies_ids = get_movies_id_list()
    n_movies = len(movies_ids)
    windows = get_windows(n_movies, J)

    n_ratings_user = len(ratings_user)
    user_vector = []

    for window in windows:

        movies_window = movies_ids[window[0] : window[1]]
        ratings_user_window = [rating for rating in ratings_user if int(rating[1]) in movies_window]
        n_ratings_user_window = len(ratings_user_window)

        user_vector.append(n_ratings_user_window) #NRW
        user_vector.append(n_ratings_user_window/n_ratings_user) #NRWR

    return user_vector

def dump_vectors_json(vectors, file):

    with open(file, 'w') as f:
        json_dumps_str = json.dumps(vectors, indent=2)
        print(json_dumps_str, file=f)
        f.close()

def obtain_genuine_vectors(J=40):

    data = load_ratings()

    users_ids = list(set(data['userId']))
    n_users = len(users_ids)
    vectors = { **dict.fromkeys([i for i in users_ids], [])} 

    for user_id in users_ids:
        ratings_user = data.loc[data["userId"] == user_id].values
        vectors[user_id] = obtain_array_user(ratings_user, 40)

    dump_vectors_json(vectors, 'vectors-genuine-users-ML10M.json')
    return vectors


def get_filler_items(filler_size, rd=np.random.RandomState(5)):
    all_items = get_movies_id_list()
    return rd.choice(all_items, replace = False, size=int(filler_size * len(all_items)))


def generate_random_average_ratings(number_users, filler_size, target_items, rating_target, type, rd):
    """
    Filler_size : percentage
    No añaden time stamp, es innecesario
    """

    ratings = [ ]

    for i in range(number_users):

        id_usuario = -1 * i
        filler_items = get_filler_items(filler_size, rd)

        if (type == 'Random'):
            mean, distribution = get_global_ratings_statistics()
        else:
            mean, distribution = get_filler_ratings_statistics(filler_items)

        filler_items_ratings = rd.normal(loc=mean, scale=distribution, size=len(filler_items))

        ratings_user = [ ]

        for item, rating in zip(filler_items, filler_items_ratings):
            ratings_user.append([id_usuario, item, rating])

        for item in target_items:
            ratings_user.append([id_usuario, item, rating_target])

        ratings.append(ratings_user)

    return(ratings)


def generate_attack_vectors_ratings(ratings, rd):

    users_ids = [i[0][0] for i in ratings]
    n_users = len(users_ids)
    vectors = { **dict.fromkeys([i for i in users_ids], [])} 

    for ratings_user in ratings:
        vectors[ratings_user[0][0]] = obtain_array_user(ratings_user, 40)

    dump_vectors_json(vectors, 'vectors-random-attack-ML10M.json')
    return vectors


def generate_attack_vectors(number, filler_size, target_items, type='Random', puntuation_range=(0, 5), push=True, rd=np.random.RandomState(5)):

    if (push):
        rating_target = puntuation_range[1]
    else:
        rating_target = puntuation_range[0]

    if type == 'Random' or type == 'Average':
        ratings = generate_random_average_ratings(number, filler_size, target_items, rating_target, type, rd)
        
    elif type == 'Bandwagon':
        return 5

    return generate_attack_vectors_ratings(ratings, rd)


In [79]:
#print(get_filler_items(0.001))

print(generate_attack_vectors(3, 0.1, [1,2,3], type='Random'))

C:\Users\patri\AppData\Local\Temp\ipykernel_10568\4167002047.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return pd.read_csv(
C:\Users\patri\AppData\Local\Temp\ipykernel_10568\4167002047.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv(
C:\Users\patri\AppData\Local\Temp\ipykernel_10568\4167002047.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv(
C:\Users\p

{0: [36, 0.03361344537815126, 26, 0.02427637721755369, 28, 0.026143790849673203, 31, 0.028944911297852476, 26, 0.02427637721755369, 30, 0.028011204481792718, 28, 0.026143790849673203, 26, 0.02427637721755369, 23, 0.021475256769374416, 23, 0.021475256769374416, 32, 0.029878618113912233, 22, 0.02054154995331466, 21, 0.0196078431372549, 25, 0.02334267040149393, 24, 0.022408963585434174, 27, 0.025210084033613446, 26, 0.02427637721755369, 26, 0.02427637721755369, 24, 0.022408963585434174, 35, 0.032679738562091505, 23, 0.021475256769374416, 37, 0.03454715219421102, 15, 0.014005602240896359, 24, 0.022408963585434174, 27, 0.025210084033613446, 27, 0.025210084033613446, 21, 0.0196078431372549, 29, 0.02707749766573296, 30, 0.028011204481792718, 25, 0.02334267040149393, 25, 0.02334267040149393, 38, 0.035480859010270774, 29, 0.02707749766573296, 21, 0.0196078431372549, 27, 0.025210084033613446, 18, 0.01680672268907563, 31, 0.028944911297852476, 30, 0.028011204481792718, 25, 0.02334267040149393, 30